In [7]:
using LinearAlgebra
using SparseArrays
using Arpack
using KrylovKit
using Dates
using JLD

## Specifying parameters of the model here:

In [16]:
L1=4;L2=4;J1=1;J2=1;h=0.01
N = L1*L2

16

## Building up Hamiltonian without using k space
- Building Hamiltonian for a generic rectangular lattice
- Using Int64 to represent states

Hamiltonian is $H=J\sum_{i=1}^{N}(S_i^z S_{i+1}^z + 1/2(S_i^+ S_{i+1}^- + S_i^- S_{i+1}^+)) $

## Building bond list (neib list) and input number of sites

### neib list for a rectangular lattice

Given the x and y dimension of the rectangular lattice (effectively, total number of sites $N$), we should output a neighbor list for all sites numbered from $1$ to $N$.

Here we are assuming we have a rectangular lattice of $L1$ in wide and $L2$ in length. So $L2$ rows and $L1$ columns in total.

In [3]:
function coordinate(n;L1::Int64=2,L2::Int64=2)
    @assert((n ≤ L1 * L2) && (1 ≤ n),"The numbering (bit position) of a site shouldn't exceed the total number of sites $(L1 * L2), and should be bigger than 0.")
    i::Int64 = Int(ceil(n/L1))
    j::Int64 = mod1(n,L1)  #site i is at i-th row, j-th column
    return (i,j)
end

function bit_pos(coordinate::Tuple{Int64,Int64};L1::Int64=2,L2::Int64=2)
    @assert((coordinate[1] ≤ L2) && (coordinate[2] ≤ L1),"The cooridnate should be within the range of the lattice size $L1 by $L2")
    n = (coordinate[1]-1)*L1 + coordinate[2]
    return n
end

bit_pos (generic function with 1 method)

### Generate neib list given that we know the dimension of the lattice

In [5]:
function nearest_neib(n::Int64;L1::Int64=2, L2::Int64=2)
    coord = coordinate(n,L1=L1, L2=L2)
    neibs = Tuple{Int64,Int64}[]
    push!(neibs, (mod1(coord[1]+1,L2), coord[2]))
    push!(neibs, (mod1(coord[1]-1,L2), coord[2]))
    push!(neibs, (coord[1], mod1(coord[2]+1,L1)))
    push!(neibs, (coord[1], mod1(coord[2]-1,L1)))    
    #=convert coordinations to positions in bits=#
    neibs_bit_pos = Set{Int64}()
    for neib in neibs
        push!(neibs_bit_pos, bit_pos(neib, L1=L1, L2=L2))
    end
    return neibs_bit_pos
end

function second_neib(n::Int64;L1::Int64=2, L2::Int64=2)
    coord = coordinate(n,L1=L1, L2=L2)
    neibs = Tuple{Int64,Int64}[]   
    if iseven(coord[1]+coord[2])
        push!(neibs, (mod1(coord[1]+1,L2), mod1(coord[2]-1,L1)))
        push!(neibs, (mod1(coord[1]-1,L2), mod1(coord[2]+1,L1)))
    else
        push!(neibs, (mod1(coord[1]+1,L2), mod1(coord[2]+1,L1)))
        push!(neibs, (mod1(coord[1]-1,L2), mod1(coord[2]-1,L1)))
    end
    #=convert coordinations to positions in bits=#
    neibs_bit_pos = Set{Int64}()
    for neib in neibs
        push!(neibs_bit_pos, bit_pos(neib, L1=L1, L2=L2))
    end
    return neibs_bit_pos
end

function nearest_neib_list_gen(;L1::Int64=2, L2::Int64=2)
    neib_list = Set{Int64}[]
    for n in 1:L1*L2
        push!(neib_list, nearest_neib(n, L1=L1, L2=L2))
    end
    return neib_list
end

function second_neib_list_gen(;L1::Int64=2, L2::Int64=2)
    neib_list = Set{Int64}[]
    for n in 1:L1*L2
        push!(neib_list, second_neib(n, L1=L1, L2=L2))
    end
    return neib_list
end

nearest_neib_list = nearest_neib_list_gen(L1=L1, L2=L2)
second_neib_list = second_neib_list_gen(L1=L1, L2=L2)

16-element Array{Set{Int64},1}:
 Set([14, 8])
 Set([7, 13])
 Set([16, 6])
 Set([5, 15])
 Set([4, 10])
 Set([9, 3]) 
 Set([2, 12])
 Set([11, 1])
 Set([16, 6])
 Set([5, 15])
 Set([14, 8])
 Set([7, 13])
 Set([2, 12])
 Set([11, 1])
 Set([4, 10])
 Set([9, 3]) 

## The Hamiltonian generator should intake bond list (neib list), number of sites and output the Hamiltonian:

So we need to revise the input of Hamiltonian function a little bit.

We still need the Hamiltonian in 2 different basis:

\begin{equation}
\tag{H1}
H=\sum_{i=1}^{N}\sum_{\sigma}JS_i^z S_{i+\sigma}^z + \frac{h}{2}\sum_{i=1}^{N} (S_i^+ + S_i^-)
\end{equation}

\begin{equation}
\tag{H2}
H=\sum_{i=1}^{N}\sum_{\sigma}\frac{J}{4}(S_i^+ + S_i^-)(S_{i+\sigma}^+ + S_{i+\sigma}^-) - h\sum_{i=1}^{N} S_i^z
\end{equation}

In [17]:
function update_val(row_inds, col_inds, vals;row_ind, col_ind, val)
    push!(row_inds, row_ind)
    push!(col_inds, col_ind)
    push!(vals, val)
end

function Hamiltonian1(;N::Int64=2, J1=1, J2=1, h=1, nearest_neib_list, second_neib_list)
    row_inds = Int64[]
    col_inds = Int64[]
    vals = Float64[]
    for state in 0:(2^N-1) #loop over all states
        state_binary = digits!(zeros(Int64, 64), state, base = 2)
        for i in 1:N #loop over all sites in a given state
            flipped_state = state ⊻ (1<<(i-1))
            update_val(row_inds, col_inds, vals, row_ind=state+1, col_ind = flipped_state+1, val = (1/2)*h)
            for j in nearest_neib_list[i] #loop over(compare) all neighbors of a given site
                update_val(row_inds, col_inds, vals, row_ind = state+1, col_ind = state+1, val = (state_binary[i]-1/2)*(state_binary[j]-1/2)*J1/2)
            end
            for j in second_neib_list[i] #loop over(compare) all neighbors of a given site
                update_val(row_inds, col_inds, vals, row_ind = state+1, col_ind = state+1, val = (state_binary[i]-1/2)*(state_binary[j]-1/2)*J2/2)
            end
        end
    end
    return sparse(row_inds, col_inds, vals, 2^N, 2^N, +)
end

function Hamiltonian2(;N::Int64=2, J1=1, J2=1, h=1, nearest_neib_list, second_neib_list)
    row_inds = Int64[]
    col_inds = Int64[]
    vals = Float64[]
    for state in 0:(2^N-1) #loop over all states
        state_binary = digits!(zeros(Int64, 64), state, base = 2)
        for i in 1:N #loop over all sites in a given state
            if state_binary[i] == 1
                update_val(row_inds, col_inds, vals, row_ind=state+1, col_ind = state+1, val = - h/2)
            else
                update_val(row_inds, col_inds, vals, row_ind=state+1, col_ind = state+1, val =  h/2)
            end
            for j in nearest_neib_list[i] #loop over(compare) all neighbors of a given site
                flipped_state = state ⊻ (1<<(i-1))
                flipped_state = flipped_state ⊻ (1<<(j-1))
                update_val(row_inds, col_inds, vals, row_ind=state+1, col_ind = flipped_state+1, val =  (1/4)*(J1/2))
            end
            for j in second_neib_list[i] #loop over(compare) all neighbors of a given site
                flipped_state = state ⊻ (1<<(i-1))
                flipped_state = flipped_state ⊻ (1<<(j-1))
                update_val(row_inds, col_inds, vals, row_ind=state+1, col_ind = flipped_state+1, val =  (1/4)*(J2/2))
            end
        end
    end
    return sparse(row_inds, col_inds, vals, 2^N, 2^N, +)
end

@time H1 = Hamiltonian1(;N=N, J1=J1, J2=J2, h=h, nearest_neib_list=nearest_neib_list, second_neib_list=second_neib_list)
@time H2 = Hamiltonian2(;N=N, J1=J1, J2=J2, h=h, nearest_neib_list=nearest_neib_list, second_neib_list=second_neib_list)

  0.476810 seconds (187.68 k allocations: 370.790 MiB, 19.79% gc time)
  0.399974 seconds (178.17 k allocations: 402.291 MiB)


65536×65536 SparseMatrixCSC{Float64,Int64} with 3211264 stored entries:
  [1    ,     1]  =  0.08
  [4    ,     1]  =  0.25
  [7    ,     1]  =  0.25
  [10   ,     1]  =  0.25
  [13   ,     1]  =  0.25
  [18   ,     1]  =  0.25
  [25   ,     1]  =  0.25
  [35   ,     1]  =  0.25
  [37   ,     1]  =  0.25
  [49   ,     1]  =  0.25
  [67   ,     1]  =  0.25
  [69   ,     1]  =  0.25
  ⋮
  [65440, 65536]  =  0.25
  [65468, 65536]  =  0.25
  [65470, 65536]  =  0.25
  [65488, 65536]  =  0.25
  [65500, 65536]  =  0.25
  [65502, 65536]  =  0.25
  [65512, 65536]  =  0.25
  [65519, 65536]  =  0.25
  [65524, 65536]  =  0.25
  [65527, 65536]  =  0.25
  [65530, 65536]  =  0.25
  [65533, 65536]  =  0.25
  [65536, 65536]  =  -0.08

## Using Lanczos to calculate the ground state and compare see if the Hamiltonians are compatible

I'm using ```KrylovKit.jl``` to calculate.

It appears that the eigenstates is automatically normalized by the package.

In [18]:
eigs1 = eigsolve(H1, 1, :SR, eltype(H1), tol = 10^(-20))
eigs2 = eigsolve(H2, 1, :SR, eltype(H2), tol = 10^(-20))
eigstate1 = eigs1[2][1]
eigstate2 = eigs2[2][1]

65536-element Array{Float64,1}:
 -0.03407968390359296   
 -5.908587333682974e-11 
  2.9287625597027225e-10
  0.011588229343780199  
  3.7495781147282075e-10
 -0.0038618322217487742 
  0.011588229205728075  
 -6.097866807360462e-11 
 -5.452759757308758e-11 
  0.011588228539578409  
 -0.0038618322607760006 
 -4.178012829361419e-11 
  0.011588229230355967  
  ⋮                     
 -4.617513732350769e-11 
 -0.004354306113600847  
  0.01306593624794893   
 -6.27225807671932e-11  
 -6.739314653909318e-11 
  0.013065936999035029  
 -0.004354306069597611  
  4.3130930299892963e-10
  0.013065937154689216  
  3.368918709948854e-10 
 -6.796562442719014e-11 
 -0.03999368288314908   

# Now we could calculate quantities given that we have the ground state.

Here x means the direction of h, y means the direction of coupling.

- $m=\frac{1}{N}\sum_{i} \langle \sigma_i^x \rangle$

Calculating this using H2

- $S_{\pi}=\frac{1}{N}\sum_{i,j}\langle\sigma_i^z\sigma_j^z\rangle (-1)^{i+j}$

Calculating this using H1

- $F_{cl}=\frac{1}{N_{pl}} \sum_{p_1,p_2} \langle f_{p_1} f_{p_2}\rangle$

Calculating this using H1

- $F_{QM}=\frac{1}{N_{pl}} \sum_{p_1,p_2} \langle o_{p_1} o_{p_2}\rangle$ with $o_p = \sigma_1^+\sigma_2^-\sigma_3^+\sigma_4^- + \sigma_1^-\sigma_2^+\sigma_3^-\sigma_4^+$

Calculating this using H1? I think

- $S_A = -Tr \rho_A \log{\rho_A}$

Where $\rho_A = Tr_B \rho$

Calculating this using either

- $Fidelity = 2\cdot\frac{1 - |\langle \psi_0(h)|\psi_0(h+\delta h)\rangle|}{\delta h^2} $

Calculating this using either

## Measuring m using H2

We basically just need to construct matrix of $m$ in the basis out which you constructed the hamiltonian H2.

In H2, $m$ is the magnetization along z direction. So:

$m=\frac{1}{N}\sum_{i} \langle \sigma_i^z \rangle$

In [22]:
function m_H2(state::Array{Float64,1}; N::Int64)
    m = spzeros(2^N,2^N)
    for basis_state in 0:(2^N-1) #loop over all states
        basis_state_binary = digits!(zeros(Int64, 64), basis_state, base = 2)
        # calculating total spin along z direction, considering it's spin 1/2
        m[basis_state+1, basis_state+1] += (sum(basis_state_binary)-1/2*N)/N
    end
    #now that we have matrix m, calculate the average m_val:
    m_val = conj.(state')*m*state
    return m_val[1] #taking the 1st value of m_val because it's recognized as a length 1 Array
end

m_H2(eigstate2; N=N)

0.005004631139658882

## Measuring $S_{\pi}$ using H1

Constructing $S_{\pi}$ matrix using basis of $H1$

$S_{\pi}=\frac{1}{N}\sum_{i,j}\langle\sigma_i^z\sigma_j^z\rangle (-1)^{i+j}$

In [26]:
function S_pi_H1(state::Array{Float64,1}; N::Int64, L1::Int64, L2::Int64)
    S_pi = spzeros(2^N,2^N)
    for basis_state in 0:(2^N-1) #loop over all states
        basis_state_binary = digits!(zeros(Int64, 64), basis_state, base = 2)
        for i in 1:N #loop over all sites in a given state
            for j in 1:N #loop over all sites again
                 S_pi[basis_state+1,basis_state+1] += (basis_state_binary[i]-1/2)*(basis_state_binary[j]-1/2)*(-1)^(sum(coordinate(i;L1=L1, L2=L2))+sum(coordinate(j;L1=L1, L2=L2)))/N
            end
        end
    end
    #now that we have matrix m, calculate the average m_val:
    S_pi_val = conj.(state')*S_pi*state
    return S_pi_val[1] #taking the 1st value of m_val because it's recognized as a length 1 Array
end

S_pi_H1(eigstate1;N=N, L1=L1, L2=L2)

0.44442496596713815

In [20]:
function S_entangle(state::Array{Float64,1}; N::Int64, L1::Int64, L2::Int64, second_neib_list)
    #find the first four spin interacting plaquette
    plaq_sites = Int64[] #Has to be an array to make it ordered
    for site in 1:N #loop over all sites
        site_coord = coordinate(site,L1=L1, L2=L2)
        if bit_pos((mod1(site_coord[1]-1, L2),mod1(site_coord[2]-1, L1)), L1=L1, L2=L2) ∈ second_neib_list[site]
            push!(plaq_sites, site)
            push!(plaq_sites, bit_pos((mod1(site_coord[1]+1, L2),site_coord[2]), L1=L1, L2=L2))
            push!(plaq_sites, bit_pos((mod1(site_coord[1]+1, L2),mod1(site_coord[2]+1, L1)), L1=L1, L2=L2))
            push!(plaq_sites, bit_pos((site_coord[1],mod1(site_coord[2]+1, L1)), L1=L1, L2=L2))
            break
        end
    end
    #println(plaq_sites)
    env_sites = Int64[]
    for site in 1:N
        if site ∉ plaq_sites
            push!(env_sites, site)
        end
    end
    #println(env_sites)
    C = zeros(Number, 16, Int(2^N/16))
    for basis_state in 0:(2^N - 1)
        basis_state_binary = digits!(zeros(Int64, 64), basis_state, base = 2)
        a = 1; b = 1 # start at 1 to avoid 0 as numeration number
        for i in 1:4
            a += basis_state_binary[plaq_sites[i]] * 2^(i-1)
        end
        for i in 1:(N-4)
            b += basis_state_binary[env_sites[i]] * 2^(i-1)
        end
        C[a, b] = state[basis_state + 1]        
    end
    Sing_vals = svd(C).S
    return  -sum(Sing_vals.^2 .* log.(Sing_vals.^2))
end

S_entangle(eigstate1;N=N, L1=L1, L2=L2, second_neib_list=second_neib_list)

1.7917594200353897

## Below we write a driver  to calculate m and $S_{\pi}$ for different h

Calculate Fidelity in the driver

In [27]:
function driver(;L1::Int64, L2::Int64, h_vals, J2)
    N = L1*L2
    J1 = 1
    nearest_neib_list = nearest_neib_list_gen(L1=L1, L2=L2)
    second_neib_list = second_neib_list_gen(L1=L1, L2=L2)
    #plaquette_list = plaquette_list_gen(;L=L, neib_list=neib_list)
    m = []
    S_pi = []
    Fidelity = []
    #Fcl_vals = []
    #Fqm_vals = []
    S_entangle_vals = []
    eigstate_prev = zeros(Float64, 2^N)
    h_prev = -1
    for i in 1:length(h_vals)
        h = h_vals[i]
        H1 = Hamiltonian1(;N=N, J1=J1, J2=J2, h=h, nearest_neib_list=nearest_neib_list, second_neib_list=second_neib_list)
        H2 = Hamiltonian2(;N=N, J1=J1, J2=J2, h=h, nearest_neib_list=nearest_neib_list, second_neib_list=second_neib_list)
        eigstate1 = eigsolve(H1, 1, :SR, eltype(H1), tol = 10^(-12))[2][1]
        eigstate2 = eigsolve(H2, 1, :SR, eltype(H2), tol = 10^(-12))[2][1]
        #calculating Fidelity using eigenstates of H1
        Fid = conj.(eigstate_prev')*eigstate1
        (i == 1) && (Fid[1] = 1) #set Fid to be 1 manually for the first h
        push!(m, m_H2(eigstate2; N=N))
        push!(S_pi, S_pi_H1(eigstate1;N=N, L1=L1, L2=L2))
        push!(Fidelity, 2*(1-abs(Fid[1]))/(h-h_prev)^2)
        #push!(Fcl_vals, Fcl(eigstate1; L=L, neib_list=neib_list, plaquette_list=plaquette_list))
        #push!(Fqm_vals, Fqm(eigstate1; L=L, neib_list=neib_list, plaquette_list=plaquette_list))
        push!(S_entangle_vals, S_entangle(eigstate1;N=N, L1=L1, L2=L2, second_neib_list=second_neib_list))
        eigstate_prev = eigstate1
        h_prev = h
    end
    #return (h_vals, m, S_pi, Fidelity, Fcl_vals, Fqm_vals, S_entangle_vals)
    return (h_vals, J2, m, S_pi, Fidelity, S_entangle_vals)
end

L1=2; L2=2; h_vals = range(0.01, 0.5, length = 50); J2_vals = range(0, 2, length = 2)
for J2 in J2_vals
    @time result = driver(L1=L1, L2=L2, h_vals=h_vals, J2=J2)

    my_time = Dates.now()

    time_finished = "Date_$(Dates.format(my_time, "e_dd_u_yyyy_HH_MM_SS"))"
    content = "Square_Spin_Ice_Measurement"
    save_path = "E:/UC Davis/Research/Square Spin Ice/Square-Spin-Ice/Yutan_code/Results/"
    #"/nfs/home/zyt329/Research/Square_spin_ice/result/"
    save_name = save_path*content*"_J2=$(J2)_L1=$(L1)_L2=$(L2)_hmin=$(h_vals[1])_hmax=$(h_vals[end])_"*time_finished*".jld"

    save(save_name, "result", result)
    println("J2 = $(J2) finished")
end

  0.090655 seconds (215.02 k allocations: 16.911 MiB, 13.14% gc time)
J2 = 0.0 finished
  0.006660 seconds (22.16 k allocations: 7.629 MiB)
J2 = 2.0 finished
